# Generate parquet for ingestion

In order to ingest data into the S2S database, it needs to follow a specific schema. This notebook will explore that schema and combine the various CSV files into a final output parquet file.

In [5]:
import sys
import os
import itertools
import boto3

import geopandas as gpd
import pandas as pd
import numpy as np

from scipy.spatial import cKDTree
from shapely.geometry import Point
from operator import itemgetter

sys.path.append("../../../gostrocks/src")

from GOSTrocks.misc import tPrint

In [6]:
good_schema_example = "s3://wbg-geography01/Space2Stats/parquet/GLOBAL/GHS_SMOD_Counts.parquet"
good_schema = pd.read_parquet(good_schema_example)
good_schema.head()

,c_11,c_12,c_13,c_21,c_22,c_23,c_30,id
0,0,0,0,0,0,0,0,86001118fffffff
1,0,0,0,0,0,0,0,8600066d7ffffff
2,0,0,0,0,0,0,0,860006c0fffffff
3,0,0,0,0,0,0,0,860010adfffffff
4,0,0,0,0,0,0,0,8600244cfffffff


The table schema for tables to be imported into Space2Stats is simple with only two columns required:
- hex_id: the h3 grid id for the polygon used in the zonal calculation; default is h3 level 6
- sum_pop_f_0_2020: attribute of interest to be ingested, should be written in the following style __f'{aggregation_method}_{attribute_name}'__

# Combine output CSV files into single parquet

In [8]:
#Search for all relevant CSV files
bucket = 'wbg-geography01' 
prefix = 'Space2Stats/h3_stats_data/GLOBAL/Urbanization_Pop/'
region = 'us-east-1'
s3client = boto3.client('s3', region_name=region, verify=False)

# Loop through the S3 bucket and get all the keys for files that are .csv 
more_results = True
loops = 0
verbose=True
good_res = []
while more_results:
    if verbose:
        print(f"Completed loop: {loops}")
    if loops > 0:
        objects = s3client.list_objects_v2(Bucket=bucket, Prefix=prefix, ContinuationToken=token)
    else:
        objects = s3client.list_objects_v2(Bucket=bucket, Prefix=prefix)
    more_results = objects['IsTruncated']
    if more_results:
        token = objects['NextContinuationToken']
    loops += 1
    for res in objects['Contents']:
        if res['Key'].endswith('.csv'):            
            cur_variable = res['Key'].split("/")[-3]
            good_res.append(res['Key'])

Completed loop: 0


c:\wbg\Anaconda3\envs\s2s\Lib\site-packages\urllib3\connectionpool.py:1099: InsecureRequestWarning: Unverified HTTPS request is being made to host 'wbg-geography01.s3.amazonaws.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


In [16]:
all_res = []
i = 0
for in_file in good_res:
    full_path = f's3://{bucket}/{in_file}'
    curD = pd.read_csv(full_path)
    break
    curD = curD.loc[:,[x for x in curD.columns if "SUM" in x]]
    curD['TOTAL_POP'] = curD.apply(lambda x: x.sum(), axis=1)
    all_res.append(curD)
    i += 1
    tPrint(f"Completed {i} of {len(good_res)}")

final_res = pd.concat(all_res)
#final_res.drop(['Unnamed: 0'], axis=1).to_parquet(f's3://{bucket}/Space2Stats/parquet/GLOBAL/GHS_SMOD_Counts.parquet')

10:09:36	Completed 1 of 842
10:09:36	Completed 2 of 842
10:09:37	Completed 3 of 842
10:09:37	Completed 4 of 842
10:09:38	Completed 5 of 842
10:09:39	Completed 6 of 842
10:09:39	Completed 7 of 842
10:09:40	Completed 8 of 842
10:09:41	Completed 9 of 842
10:09:41	Completed 10 of 842
10:09:42	Completed 11 of 842
10:09:42	Completed 12 of 842
10:09:43	Completed 13 of 842
10:09:43	Completed 14 of 842
10:09:44	Completed 15 of 842
10:09:44	Completed 16 of 842
10:09:45	Completed 17 of 842
10:09:45	Completed 18 of 842
10:09:46	Completed 19 of 842
10:09:47	Completed 20 of 842
10:09:47	Completed 21 of 842
10:09:48	Completed 22 of 842
10:09:48	Completed 23 of 842
10:09:49	Completed 24 of 842
10:09:50	Completed 25 of 842
10:09:50	Completed 26 of 842
10:09:51	Completed 27 of 842
10:09:51	Completed 28 of 842
10:09:52	Completed 29 of 842
10:09:52	Completed 30 of 842
10:09:53	Completed 31 of 842
10:09:54	Completed 32 of 842
10:09:54	Completed 33 of 842
10:09:55	Completed 34 of 842
10:09:56	Completed 35 o

In [22]:
curD.head()

,11_SUM,12_SUM,13_SUM,21_SUM,22_SUM,23_SUM,30_SUM,TOTAL_POP
0,-1.0,0.0,0.0,0.0,0.0,0.0,0.0,-1.0
1,-1.0,0.0,0.0,0.0,0.0,0.0,0.0,-1.0
2,-1.0,0.0,0.0,0.0,0.0,0.0,0.0,-1.0
3,-1.0,0.0,0.0,0.0,0.0,0.0,0.0,-1.0
4,-1.0,0.0,0.0,0.0,0.0,0.0,0.0,-1.0


In [20]:
good_schema.head()

,c_11,c_12,c_13,c_21,c_22,c_23,c_30,id
0,0,0,0,0,0,0,0,86001118fffffff
1,0,0,0,0,0,0,0,8600066d7ffffff
2,0,0,0,0,0,0,0,860006c0fffffff
3,0,0,0,0,0,0,0,860010adfffffff
4,0,0,0,0,0,0,0,8600244cfffffff


In [21]:
final_res.head()

,11_SUM,12_SUM,13_SUM,21_SUM,22_SUM,23_SUM,30_SUM,TOTAL_POP
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [19]:
# For the population data, merge the GHS SMOD counts and the population data
all_data = pd.mergea

(14117882, 8)

In [18]:
final_res.shape

(14117882, 8)